# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**



In [1]:
!pip install dlt[duckdb] # Install dlt with all the necessary DuckDB dependencies

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.7/540.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.2 MB/s eta 0:00:00


In [4]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

sum=0
i=1

for sqrt_value in generator:
    print("The",i,"th value is",sqrt_value)
    sum=sum+sqrt_value
    if i==5:
      print("The sum of the first 5 values are", sum)
    i=i+1



The 1 th value is 1.0
The 2 th value is 1.4142135623730951
The 3 th value is 1.7320508075688772
The 4 th value is 2.0
The 5 th value is 2.23606797749979
The sum of the first 5 values are 8.382332347441762
The 6 th value is 2.449489742783178
The 7 th value is 2.6457513110645907
The 8 th value is 2.8284271247461903
The 9 th value is 3.0
The 10 th value is 3.1622776601683795
The 11 th value is 3.3166247903554
The 12 th value is 3.4641016151377544
The 13 th value is 3.605551275463989


Based on the above output, the sum of the first  5 values is 8.38 while the output of the 13 th one is 3.605

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [6]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [16]:
import dlt
import duckdb
#the piple
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='city_data')


# load th first generator
info = generators_pipeline.run(people_1(),
										table_name="individual_data",
										write_disposition="replace")

#append the second generator to the first ome

info =generators_pipeline.run( people_2(),
										table_name="individual_data",
										write_disposition="append")


#Connect to the pipeline


conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# connect to the dataset that the table is saved at
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")




In [17]:
#convert the table to the dataframe
passengers = conn.sql("SELECT * FROM individual_data").df()

In [18]:
passengers.head(10)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707943778.5225136,YIwjOznMXWI3+w,None
1,2,Person_2,27,City_A,1707943778.5225136,l11uVz49lzKtGw,None
2,3,Person_3,28,City_A,1707943778.5225136,xuIJ/W8Hcfv7yA,None
3,4,Person_4,29,City_A,1707943778.5225136,ircrDlvdJzsWjA,None
4,5,Person_5,30,City_A,1707943778.5225136,K3gqFGnJT4xe+w,None
5,3,Person_3,33,City_B,1707943779.3329194,nof+gFY5eeKBfw,Job_3
6,4,Person_4,34,City_B,1707943779.3329194,2MiSIhktW+Medw,Job_4
7,5,Person_5,35,City_B,1707943779.3329194,JudArjaxVT1UuQ,Job_5
8,6,Person_6,36,City_B,1707943779.3329194,iUZT1gC5cpID/w,Job_6
9,7,Person_7,37,City_B,1707943779.3329194,BmB3KFVAa1NT6w,Job_7


In [19]:
#get the sum of the ages
passengers["age"].sum()

353

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [24]:
#the piple
generators_pipeline_merge = dlt.pipeline(destination='duckdb', dataset_name='city_data')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline_merge.run(people_1(),
										table_name="individual_data",
										write_disposition="replace")

#merge the data from the second generator using the id as the primary id
info =generators_pipeline_merge.run( people_2(),
										table_name="individual_data",
										write_disposition="merge",   primary_key="id")


#Connect to the pipeline


conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")


conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")


In [25]:
passengers = conn.sql("SELECT * FROM individual_data").df()

In [26]:
passengers.head(10)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707943938.4834044,FycpyWjOce6h4A,None
1,2,Person_2,27,City_A,1707943938.4834044,t7KSmhISEqBnoA,None
2,5,Person_5,35,City_B,1707943939.153677,KqJzBR3pzijjBg,Job_5
3,8,Person_8,38,City_B,1707943939.153677,brs2G0VAIw8Pyg,Job_8
4,7,Person_7,37,City_B,1707943939.153677,yoRJuMhFV5vMDg,Job_7
5,4,Person_4,34,City_B,1707943939.153677,ethFLZKs4Z30jA,Job_4
6,3,Person_3,33,City_B,1707943939.153677,0pwOu7FS/z/hjg,Job_3
7,6,Person_6,36,City_B,1707943939.153677,vm1ODxFXjxlfwg,Job_6


In [27]:
len(passengers)

8

In [28]:
#Get the sum of the table after merging the two generators
passengers["age"].sum()

266